In [7]:
import pandas as pd 
import numpy as np
import seaborn as sns

import base64
import requests

## Importing, curating and exploring historic seismic data from SSN

In [9]:
#Data retrieved from Mexican Seismological Service from UNAM (http://www2.ssn.unam.mx:8080/catalogo/#)
#Data contains 262,782 rows with data from 1900 to September 2022 with historic seisms detection

#data = pd.read_csv('SSNMX_19000101_20221012.csv')

url = 'https://github.com/eflopezruiz/19SeptemberCDMX/blob/64b15e645f4e4851d1d1f49c1602f61827aa5b2f/SSNMX_19000101_20221012.csv?raw=true'
data = pd.read_csv(url)
print(data.shape)
print(data.info())
data.head(3)

C:\Users\edlopezr\AppData\Local\Temp\ipykernel_15012\2515337472.py:7: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url)


(262782, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262782 entries, 0 to 262781
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Fecha                       262782 non-null  object 
 1   Hora                        262782 non-null  object 
 2   Magnitud                    262782 non-null  object 
 3   Latitud                     262782 non-null  float64
 4   Longitud                    262782 non-null  float64
 5   Profundidad                 262782 non-null  object 
 6   Referencia de localizacion  262782 non-null  object 
 7   Fecha UTC                   262782 non-null  object 
 8   Hora UTC                    262782 non-null  object 
 9   Estatus                     262782 non-null  object 
dtypes: float64(2), object(8)
memory usage: 20.0+ MB
None


,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,1/20/1900,0:33:30,7.4,20.0,-105.0,33,"71 km al NOROESTE de AUTLAN DE NAVARRO, JAL",1/20/1900,6:33:30,revisado
1,5/16/1900,14:12:00,6.9,20.0,-105.0,33,"71 km al NOROESTE de AUTLAN DE NAVARRO, JAL",5/16/1900,20:12:00,revisado
2,3/5/1901,4:45:00,6.9,25.0,-110.0,33,"100 km al NORESTE de LA PAZ, BCS",3/5/1901,10:45:00,revisado
